### Ultrahack 2016 MyData - HSL:n kaupunkipyörät
    - HSL:n kaupunkipyörien kysynnän ennustaminen asema- ja tuntikohtaisesti
    - Perustuen:
        - Kaupunkipyöräpisteiden kapasiteettilokiin
        - Helsingin avoimeen säädataan

In [1]:
library(dplyr)
library(ggplot2)
library(geosphere)
library(caret)
library(ranger)
library(e1071)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

Loading required package: sp
Loading required package: lattice


In [2]:
bikedata <- read.table('/users/ikonhen/omat/ultrahack/hsl_bike_data.csv', 
                       header = T, sep = '\t', stringsAsFactors = F)

weatherdata <- read.table('/users/ikonhen/omat/ultrahack/weather_data_combined.csv', 
                          header = T, sep = '\t', stringsAsFactors = F)
head(bikedata, 3)
head(weatherdata, 3)

,timestamp,name,operative,style,lat,lon,total_slots,free_slots,avl_bikes
1,20160426T141326Z,A21 Varsapuistikko,True,,60.173099,24.949636,28,28,0
2,20160426T141326Z,B03 Haapaniemenkatu,True,,60.181907,24.95688,18,18,0
3,20160426T141326Z,B08 Sörnäisten metroasema,True,,60.187704,24.960505,24,24,0


,TimeEEST,TemperatureC,Dew.PointC,Humidity,Sea.Level.PressurehPa,VisibilityKm,Wind.Direction,Wind.SpeedKm.h,Gust.SpeedKm.h,Precipitationmm,Events,Conditions,WindDirDegrees,DateUTC
1,12:20 AM,-1,-1,100,1006,-9999,WNW,3.7,-,NA,,Clear,300,20160331T212000Z
2,12:50 AM,1,0,93,1006,10,West,9.3,-,NA,,Mostly Cloudy,280,20160331T215000Z
3,1:20 AM,1,0,93,1006,-9999,WNW,9.3,-,NA,,Overcast,290,20160331T222000Z


 **Aggregoidaan datasetit:**
     - sekä HSL:n pyörä-, että yleinen Helsingin säädatasetti
     - aikamuuttujan konversio
     - jatkuvien aggregointi tuntitasolle
     - kevyttä piirrelaskentaa

In [3]:
bikedata %>%

    tbl_df %>%
    mutate(station = substr(name, 1 ,3),
           month = as.numeric(substr(timestamp, 5, 6)),
           dayofmonth = as.numeric(substr(timestamp, 7, 8)),
           dayofweek = as.POSIXlt(as.Date(substr(timestamp, 1, 8), format = '%Y%m%d'))$wday,
           hour = as.numeric(substr(timestamp, 10, 11)),
           minute = as.numeric(substr(timestamp, 12, 13))) %>%
    select(-name, -timestamp, -style, -minute) %>%
    group_by(station, month, lat, lon, total_slots, dayofmonth, hour, operative, dayofweek) %>%
    summarise(avg_availability = round(mean(avl_bikes)),
              max_availability = max(avl_bikes)) %>%
    mutate(hour_capacity_mean = round(avg_availability / total_slots)*100,
           hour_capacity_max = round(max_availability / total_slots)*100) -> bikedata

invisible(gc())
head(bikedata, 3)

,station,month,lat,lon,total_slots,dayofmonth,hour,operative,dayofweek,avg_availability,max_availability,hour_capacity_mean,hour_capacity_max
1,A01,4,60.155411,24.950391,30,28,5,True,4,0,0,0,0
2,A01,4,60.155411,24.950391,30,28,8,True,4,0,0,0,0
3,A01,4,60.155411,24.950391,30,28,9,True,4,0,0,0,0


In [4]:
getmode <- function(v) {
   uniqv <- unique(v)
   return(uniqv[which.max(tabulate(match(v, uniqv)))])
}

weatherdata[, c('DateUTC', 'TemperatureC', 'Humidity', 'Wind.SpeedKm.h', 'Conditions')] %>%

    tbl_df %>%
    mutate(month = as.numeric(substr(DateUTC, 5, 6)),
           dayofmonth = as.numeric(substr(DateUTC, 7, 8)),
           dayofweek = as.POSIXlt(as.Date(substr(DateUTC, 1, 8), format = '%Y%m%d'))$wday,
           hour = as.numeric(substr(DateUTC, 10, 11)),
           minute = as.numeric(substr(DateUTC, 12, 13)),
           cond_code = as.numeric(as.factor(Conditions)),
           windspeed = as.numeric(Wind.SpeedKm.h)) %>%
    select(-DateUTC, -minute, -Conditions, -Wind.SpeedKm.h) %>%
    group_by(month, dayofmonth, dayofweek, hour) %>%
    summarise(avg_temp = mean(TemperatureC), 
              avg_humidity = mean(Humidity),
              avg_windspeed = mean(windspeed),
              hour_condition = getmode(cond_code)) -> weatherdata

head(weatherdata, 3)

Warning message:
In eval(substitute(expr), envir, enclos): NAs introduced by coercion

,month,dayofmonth,dayofweek,hour,avg_temp,avg_humidity,avg_windspeed,hour_condition
1,3.0,31.0,4.0,21.0,0.0,96.5,6.5,1.0
2,3.0,31.0,4.0,22.0,1.0,93.0,10.2,15.0
3,3.00,31.00,4.00,23.00,1.00,93.00,12.05,15.00


**Piirrelaskenta jatkuu:**
    - Asemien koordinaatit
    - Etäisyys lähimpään naapuriasemaan
    - Lähimmät kolme naapuriasemaa (asemakoodit)

In [5]:
station_info <- data.frame(station = unique(bikedata$station),
                           min_neighbor_dist = 0,
                           neighbor_1 = 'empty',
                           neighbor_2 = 'empty',
                           neighbor_3 = 'empty',
                           latitude = 0, 
                           longitude = 0)

factors <- sapply(station_info, is.factor)
station_info[factors] <- lapply(station_info[factors], as.character)                 

for (station in station_info$station) {
    stnumber <- grep(station, station_info$station)
    station_info[stnumber, c('latitude')] <- max(bikedata$lat[bikedata$station == station])
    station_info[stnumber, c('longitude')] <- max(bikedata$lon[bikedata$station == station])
}

head(station_info, 3)

,station,min_neighbor_dist,neighbor_1,neighbor_2,neighbor_3,latitude,longitude
1,A01,0,empty,empty,empty,60.155411,24.950391
2,A02,0,empty,empty,empty,60.159715,24.955212
3,A03,0,empty,empty,empty,60.158172,24.944808


In [6]:
for (station in station_info$station) {
    stnumber <- grep(station, station_info$station)
    other_stations <- station_info$station[-stnumber]
    distance <- c()
    results <- data.frame()
    i <- 1
    for (comparison in other_stations) {
        tryCatch({
            cmnumber <- grep(comparison, station_info$station)
            station_point <- station_info[stnumber, c('latitude', 'longitude')]
            comparison_point <- station_info[cmnumber, c('latitude', 'longitude')]
            distance[i] <- distHaversine(station_point, comparison_point)
            i <- i + 1
            }, error = function(e) { cat(paste("ERROR :", conditionMessage(e), station, comparison), "\n") })
    }
    results <- data.frame(station_1 = station, station_2 = other_stations, distance = distance)
    results <- results[order(distance), ]
    station_info[stnumber, c('min_neighbor_dist')] <- results[1, 3]
    station_info[stnumber, c('neighbor_1')] <- as.character(results[1, 2])
    station_info[stnumber, c('neighbor_2')] <- as.character(results[2, 2])
    station_info[stnumber, c('neighbor_3')] <- as.character(results[3, 2])
}

head(station_info, 3)

,station,min_neighbor_dist,neighbor_1,neighbor_2,neighbor_3,latitude,longitude
1,A01,681.115288302423,A03,A02,A07,60.155411,24.950391
2,A02,690.446191559871,A01,A13,A11,60.159715,24.955212
3,A03,396.897591474481,A04,A06,A07,60.158172,24.944808


**Aineistojen yhdistely**
    - huhtikuun datat filtteröidään pois
    - tyhjien ja negatiivisten arvojen konversio nolliksi
    - char- muuttujien koodaus int- muotoon

In [8]:
include <- c('station', 'min_neighbor_dist', 'neighbor_1', 'neighbor_2', 'neighbor_3')

bikedata %>%

    left_join(station_info[, include], by = 'station', all.x = T) %>%
    left_join(weatherdata, by = c('month', 'dayofmonth', 'hour'), all.x = T) %>%
    select(-dayofweek.y, -avg_availability, -hour_capacity_max, -hour_capacity_mean) %>%
    filter(month > 4) -> training_data

training_data[is.na(training_data)] <- 0
training_data[training_data < 0] <- 0

for (feature in colnames(training_data)) {
    if (class(training_data[[feature]]) == 'character') {
        training_data[[feature]] <- as.numeric(as.factor(training_data[[feature]]))
    } 
}

head(training_data, 3)

,station,month,lat,lon,total_slots,dayofmonth,hour,operative,dayofweek.x,max_availability,min_neighbor_dist,neighbor_1,neighbor_2,neighbor_3,avg_temp,avg_humidity,avg_windspeed,hour_condition
1,1.00000,5.00000,60.15541,24.95039,30.00000,1.00000,8.00000,1.00000,0.00000,0.00000,681.11529,2.00000,1.00000,4.00000,14.00000,55.00000,11.10000,1.00000
2,1.00000,5.00000,60.15541,24.95039,30.00000,2.00000,6.00000,1.00000,1.00000,0.00000,681.11529,2.00000,1.00000,4.00000,12.50000,44.00000,4.65000,1.00000
3,1.00000,5.00000,60.15541,24.95039,30.00000,2.00000,7.00000,1.00000,1.00000,0.00000,681.11529,2.00000,1.00000,4.00000,14.00000,35.50000,8.35000,1.00000


In [ ]:
rf_control <- trainControl(method = 'cv', number = 5, allowParallel = TRUE, verbose = T)

parallel_rf_search <- train(max_availability ~.,
                            data = training_data,
                            method = "ranger",
                            trControl = rf_control,
                            tuneGrid = expand.grid(mtry = c(3, 4, 5, 6)),
                            num.trees = 200)
